In [ ]:
import numpy as np
import argparse

import os
import math
from math import sin

import pandas as pd
from sklearn.linear_model import LinearRegression

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.offline as pyo
import plotly.graph_objs as go

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
%matplotlib inline

i_seed = 0

import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PatchWand/') # add this line so Data and data are visible in this file

# from PatchWand import *
from filters import *
from setting import *
from plotting_tools import *
# from stage1_patch import *
# from TimeStampReader import *
# from Subject import *
# from PatchParser import *
# from preprocessing import *
# from ECG_module import *
# from PPG_module import *
# from segmentation import *
# from stage3_inspection import *


from importlib import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import scipy.io
mat = scipy.io.loadmat(inputdir)

In [ ]:
# mat

In [ ]:
inputdir = '../../data/raw/sub104/watch/patchdataSet.mat'

In [ ]:
for dirname in inputdir.split('/'):
    if 'sub' not in dirname:
        continue
    subject_id = dirname
subject_id

In [ ]:
FS_RESAMPLE

In [ ]:
def data_interpolation2(raw_dict, subject_id, FS_RESAMPLE=500):

    ecg_time = raw_dict['E_time'].squeeze()
    ppg_time = raw_dict['ppgF_time'].squeeze()

    t_start = np.max([ecg_time[0], ppg_time[0]])
    t_end = np.min([ecg_time[-1], ppg_time[-1]])

    time_interp = np.arange(my_ceil(t_start, decimal=-3)*FS_RESAMPLE, my_floor(t_end, decimal=-3)*FS_RESAMPLE+1)/FS_RESAMPLE
    
    patch_dict = {}

    # ECG
    patch_dict['ECG'] = np.interp(time_interp, raw_dict['E_time'].squeeze(), raw_dict['E_data'].squeeze())

#     patch_dict['ECG'] = get_padded_filt(patch_dict['ECG'], filter_padded=5, lowcutoff=FILT_ECG[0], highcutoff=FILT_ECG[1], Fs=FS_RESAMPLE)

    
    # TODO: find out which PPG arr this belongs to (in terms of its physical location)
    
    # PPG array 1
    patch_dict['ppg_g_1'] = np.interp(time_interp, raw_dict['ppgF_time'].squeeze(), raw_dict['ppgF_G1'].squeeze())
    patch_dict['ppg_r_1'] = np.interp(time_interp, raw_dict['ppgF_time'].squeeze(), raw_dict['ppgF_R1'].squeeze())
    patch_dict['ppg_ir_1'] = np.interp(time_interp, raw_dict['ppgF_time'].squeeze(), raw_dict['ppgF_I1'].squeeze())    
 
    patch_dict['ppg_g_1'] = -get_padded_filt(patch_dict['ppg_g_1'], filter_padded=5, lowcutoff=FILT_PPG[0], highcutoff=FILT_PPG[1], Fs=FS_RESAMPLE)
    patch_dict['ppg_ir_1'] = -get_padded_filt(patch_dict['ppg_ir_1'], filter_padded=5, lowcutoff=FILT_PPG[0], highcutoff=FILT_PPG[1], Fs=FS_RESAMPLE)
    patch_dict['ppg_r_1'] = -get_padded_filt(patch_dict['ppg_r_1'], filter_padded=5, lowcutoff=FILT_PPG[0], highcutoff=FILT_PPG[1], Fs=FS_RESAMPLE)
    

    # PPG array 2
    patch_dict['ppg_g_2'] = np.interp(time_interp, raw_dict['ppgF_time'].squeeze(), raw_dict['ppgF_G2'].squeeze())
    patch_dict['ppg_r_2'] = np.interp(time_interp, raw_dict['ppgF_time'].squeeze(), raw_dict['ppgF_R2'].squeeze())
    patch_dict['ppg_ir_2'] = np.interp(time_interp, raw_dict['ppgF_time'].squeeze(), raw_dict['ppgF_I2'].squeeze())    
 
    patch_dict['ppg_g_2'] = -get_padded_filt(patch_dict['ppg_g_2'], filter_padded=5, lowcutoff=FILT_PPG[0], highcutoff=FILT_PPG[1], Fs=FS_RESAMPLE)
    patch_dict['ppg_ir_2'] = -get_padded_filt(patch_dict['ppg_ir_2'], filter_padded=5, lowcutoff=FILT_PPG[0], highcutoff=FILT_PPG[1], Fs=FS_RESAMPLE)
    patch_dict['ppg_r_2'] = -get_padded_filt(patch_dict['ppg_r_2'], filter_padded=5, lowcutoff=FILT_PPG[0], highcutoff=FILT_PPG[1], Fs=FS_RESAMPLE)
    # PPG array 3
    patch_dict['ppg_g_3'] = np.interp(time_interp, raw_dict['ppgF_time'].squeeze(), raw_dict['ppgF_G3'].squeeze())
    patch_dict['ppg_r_3'] = np.interp(time_interp, raw_dict['ppgF_time'].squeeze(), raw_dict['ppgF_R3'].squeeze())
    patch_dict['ppg_ir_3'] = np.interp(time_interp, raw_dict['ppgF_time'].squeeze(), raw_dict['ppgF_I3'].squeeze())
 
    patch_dict['ppg_g_3'] = -get_padded_filt(patch_dict['ppg_g_3'], filter_padded=5, lowcutoff=FILT_PPG[0], highcutoff=FILT_PPG[1], Fs=FS_RESAMPLE)
    patch_dict['ppg_ir_3'] = -get_padded_filt(patch_dict['ppg_ir_3'], filter_padded=5, lowcutoff=FILT_PPG[0], highcutoff=FILT_PPG[1], Fs=FS_RESAMPLE)
    patch_dict['ppg_r_3'] = -get_padded_filt(patch_dict['ppg_r_3'], filter_padded=5, lowcutoff=FILT_PPG[0], highcutoff=FILT_PPG[1], Fs=FS_RESAMPLE)

#     # skin temperature
#     patch_dict['temp_skin'] = np.interp(time_interp, raw_dict['env_time'], medfilt(raw_dict['temp_skin'], k=3))
#     # pressure
#     patch_dict['pres'] = np.interp(time_interp, raw_dict['env_time'], get_smooth(raw_dict['pres'], N=51))

    time_interp = time_interp-time_interp[0]
    patch_dict['time'] = time_interp
    patch_dict['subject_id'] = subject_id

    return patch_dict

def my_ceil(arr, decimal=0):
    return np.ceil(arr*(10**-decimal))/(10**-decimal)

def my_floor(arr, decimal=0):
    return np.floor(arr*(10**-decimal))/(10**-decimal)



In [ ]:
watch_dict = data_interpolation2(mat, subject_id, FS_RESAMPLE=FS_RESAMPLE)
df_watch = pd.DataFrame(watch_dict)


In [ ]:
df_watch

In [ ]:
mat

In [ ]:
# with open('patch_tap_times.json', 'r') as f:
#     patch_tap_times = json.load(f)[0]


# if subject_id not in patch_tap_times:
# if subject_id in patch_tap_times:
# Create traces
fig = make_subplots(specs=[[{'secondary_y': True}]])

t_start = 0
# t_start = df_watch['time'].max()-20*60
t_end = df_watch['time'].max()-60*60
df = df_watch[(df_watch['time']>t_start) & (df_watch['time']<t_end)] # ignore the last 60minutes because the study is at least 60 minute long prior to stop recording
downsample_factor = 1


alpha = 0.5

v_ECG = df['ECG'].values
# v_ppg_g_1 = df['ppg_g_1'].values
# v_ppg_ir_1 = df['ppg_ir_1'].values
# v_ppg_r_1 = df['ppg_r_1'].values
v_ppg_g_1 = df['ppg_g_3'].values
v_ppg_ir_1 = df['ppg_ir_3'].values
v_ppg_r_1 = df['ppg_r_3'].values

ts = df['time'].values


fig.add_trace(go.Scatter(x=mat['E_time'].squeeze(), y=mat['E_data'].squeeze(),
                    mode='lines',
                    name='ECG',
                         opacity=alpha,
                      line=dict(color=color_dict[sync_color_dict['ECG']])),
              secondary_y=False,)
# fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ECG[::downsample_factor],
#                     mode='lines',
#                     name='ECG',
#                          opacity=alpha,
#                       line=dict(color=color_dict[sync_color_dict['ECG']])),
#               secondary_y=False,)

# fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ppg_g_1[::downsample_factor],
#                     mode='lines',
#                     name='ppg_g_1',
#                          opacity=alpha,
#                       line=dict(color=color_dict[sync_color_dict['ppg_g_1']])),
#               secondary_y=False,)

# fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ppg_ir_1[::downsample_factor],
#                     mode='lines',
#                     name='ppg_ir_1',
#                          opacity=alpha,
#                       line=dict(color=color_dict[sync_color_dict['ppg_ir_1']])),
#               secondary_y=False,)

# fig.add_trace(go.Scatter(x=ts[::downsample_factor], y=v_ppg_r_1[::downsample_factor],
#                     mode='lines',
#                     name='ppg_r_1',
#                          opacity=alpha,
#                       line=dict(color=color_dict[sync_color_dict['ppg_r_1']])),
#               secondary_y=False,)


# Add figure title
fig.update_layout(
    title_text="inspect taps"
)

# Set y-axes titles
fig.update_yaxes(title_text="<b>signal (a.u.)</b>", secondary_y=False)

fig.update_xaxes(title_text="<b>time (sec)</b>")
fig.update_layout(plot_bgcolor='rgb(255,255,255)')

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)


fig.show()
# sys.exit()
# else:
# tap_time = patch_tap_times[subject_id]
# print(tap_time)


In [ ]:
ppgS_I1 = mat['ppgS_I1']

In [ ]:
mat['ppgS_time']